## Import Dependencies

Instructions for SUTime (Taken from this page: https://github.com/FraBle/python-sutime)

To install SUTime, follow these steps:
```python
# Ideally, create a virtual environment before installing any dependencies
pip install sutime
# Install Java dependencies
mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python3 -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec("sutime").origin).parent / "pom.xml")')
```

In [2]:
import numpy as np
import pandas as pd
import openai
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import re
import tiktoken
import time
import faiss
import awoc
import spacy
import csv
nlp = spacy.load("en_core_web_sm")
from sklearn.metrics.pairwise import cosine_similarity
#from sutime import SUTime
import json
import datetime
from bert_score import score as bert_score
from country_named_entity_recognition import find_countries
import ast
import datetime

import sys
sys.path.insert(1, '../utils')



/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Raw Documents Data

In [20]:
# main data
wdi_csv = pd.read_csv('../data/WDI_CSV/WDICSV.csv')
# country meta data
wdi_country = pd.read_csv('../data/WDI_CSV/WDICountry.csv')
# Series meta data
wdi_series = pd.read_csv('../data/WDI_CSV/WDISeries.csv')
# country + series
#wdi_country_series = pd.read_csv('../data/WDI_CSV/WDIcountry-series.csv')
# series + time
#wdi_series_time = pd.read_csv('../data/WDI_CSV/WDIseries-time.csv')
# With CountryCode + SeriesCode + year, describe more info about this resource
#wdi_footnote = pd.read_csv('../data/WDI_CSV/WDIfootnote.csv')

## Load Environments

In [21]:
load_dotenv()

True

## OpenAI API Configuration

In [22]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

encoding = tiktoken.get_encoding('cl100k_base')
embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

In [17]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response

To get any information from WDICSV.csv (WDI meta data) we need 3 things: 1. country code 2. indicator code 3. target period (1960 - 2023)

## Function for searching country code (First Condition Done ✅)

In [24]:
'''
Previous 'find_mentioned_countries' can detect countries when they are formed correctly.

'''
# Extract mentioned countries' ISO3 code
def find_mentioned_country_code(user_query):
    countries = set()
    extracted_countries = find_countries(user_query, is_ignore_case=True)
    # check if we have country first
    if extracted_countries:
        for c in extracted_countries:
            countries.add(c[0].alpha_3)
    # check if we have continent
    else:
        words = re.findall(r'\w+|[^\w\s]', user_query)
        text = ' '.join(words)  # Join the tokens back into a string
    
        world_info = awoc.AWOC()
        all_continents = set([continent.lower() for continent in world_info.get_continents_list()])
        for word in text.split():
            word = word.lower()
            # check if this continent
            if word in all_continents:
                target_countries = world_info.get_countries_list_of(word)
                for country in target_countries:
                    countries.add(world_info.get_country_data(country)['ISO3'])
    return countries

# Function for searching indicator code (Second Condition Done✅)

## Embedding Processing for Indicators

In [26]:
def create_embedding(row):
    time.sleep(3)
    #print(row.name)
    input_text = row['Indicator Name'].replace("\n", " ")
    input_text = re.sub(r'\s+', ' ', input_text)
    encodings = encoding.encode(input_text)
    length = len(encodings)
    embedding = client.embeddings.create( 
        input=input_text ,model= embedding_model
    ).data[0].embedding
    
    return length, embedding

#wdi_series['token_length'], wdi_series['Embedding'] = zip(*wdi_series.apply(lambda row: create_embedding(row), axis=1))

In [8]:
#wdi_series.to_pickle('../data/indicator_meta_embed.pkl')

## Searching target indicator

In [27]:
df = pd.read_pickle('../models/indicator_meta_embed.pkl')

In [28]:
# Function to calculate Jaccard similarity between two texts
def jaccard_similarity(text1, text2):
    # Tokenize texts
    tokens1 = set(text1.lower().split())
    tokens2 = set(text2.lower().split())
    
    # Calculate Jaccard similarity
    intersection = len(tokens1.intersection(tokens2))
    union = len(tokens1.union(tokens2))
    
    return intersection / union if union > 0 else 0


In [29]:
def filter_indicators(user_query):
    # Calculate similarity scores for each indicators
    similarity_scores = []
    indicators = []

    # Iterate through each indicator title and calculate similarity score
    for indicator in df['Indicator Name']:
        similarity_score = jaccard_similarity(user_query, indicator)
        similarity_scores.append(similarity_score)
        indicators.append(indicator)
        
    # Create DataFrame only with valid similarity scores
    similarity_df = pd.DataFrame({'Indicator Name': indicators, 'Similarity Score': similarity_scores})
    similarity_df = similarity_df.sort_values('Similarity Score', ascending=False)
    similarity_df = similarity_df[:10]
        
    # Filter indicators where similarity score is above a threshold (e.g., 0.3)
    threshold = 0.01
    filtered_df = df[df['Indicator Name'].isin(similarity_df[similarity_df['Similarity Score'] > threshold]['Indicator Name'])]

    return  list(filtered_df['Series Code'])
#print(filter_indicators(test_query))
#print(filter_indicators(test_query2))

In [30]:
# search target indicator
# Implement this function later
def search_embeddings(user_query):
    df_filtered = filter_indicators(user_query) if filter_indicators(user_query) is not None else None
    
    if df_filtered is not None and not df_filtered.empty:  # Check if DataFrame is not None and not empty
        length = len(df_filtered.head())
        filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
        index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
        index.add(filtered_embeddings_arrays)
        
        user_query_embedding = client.embeddings.create( 
                input=user_query ,model= embedding_model
            ).data[0].embedding

        k = min(5, length)
        distances, indices = index.search(np.array([user_query_embedding]), k)
        return df_filtered, distances, indices
    else:
        return None, None, None

# Function for searching target period (1960 - 2023) (Third Condition)

First Trial - Package is too heavy❌

In [31]:
# Extract set of years from given timex3_list
def timex3_to_year_list(timex3_list):
    year_list = set()
    for timex3 in timex3_list:
        sutimeType, value = timex3["type"], timex3["value"]
        if "REF" not in value:
            if isinstance(value, dict):
                if value:
                    for year in range(int(value['begin']), int(value['end']) + 1):
                        year_list.add(str(year))
            elif value.isdigit():
                year_list.add(str(value))
            elif sutimeType in ['DATE', 'DURATION']:
                if sutimeType == 'DATE':
                    res = re.search('^\d\d\d\d', value)
                    if res:
                        year_list.add(str(res.group(0)))
                else:
                    year_dur = 0
                    current_year = datetime.now().year
                    dur_list = re.findall('\d+', "".join(re.findall('P[0-9]+Y', value)))
                    if dur_list:
                        year_dur = max([int(y) for y in dur_list])
                        while year_dur:
                            year_list.add(str(current_year - year_dur))
                            year_dur -= 1
            else:
                continue
    return list(year_list)
    
def find_target_period(user_query):
    sutime = SUTime(mark_time_ranges = True, include_range = True)
    res = sutime.parse(user_query)
    return timex3_to_year_list(res)

Second Trial - using GPT. Works pretty well ✅

In [32]:
def find_target_period(user_query):
    current_year = datetime.date.today().year
    gpt_prompt = f"""
    Identify and extract all the years mentioned in the provided user query, returning them as a list. Current year is {current_year}
    The format of the output should be a list of strings, each representing a year, e.g., [\"2020\", \"2021\"].

    User Query: {user_query}
    """

    answer = callOpenAI(gpt_prompt)

    year_list = ast.literal_eval(answer)
    
    return year_list if year_list else []

In [33]:
test1 = "past 5 years"
test2 = "in 2021"
test3 = "from 2020 to 2023"
print(find_target_period(test1))
print(find_target_period(test2))
print(find_target_period(test3))

['2019', '2020', '2021', '2022', '2023']
['2021']
['2020', '2021', '2022', '2023']


## Final one function for searching indicator data (Function for finding info from indicator database)

In [34]:
def map_to_structure(countries, indicators, years):
    # load all indicator dataset
    # wdi_csv = pd.read_csv('../data/WDI_CSV/WDICSV.csv')
    count = 0
    result_dict = {}
    for country in countries:
        for indicator in indicators:
            indicator_id = f"indicator-{count + 1}"
            target_row = wdi_csv[(wdi_csv['Country Code'] == country) & (wdi_csv['Indicator Code'] == indicator)]
            if not target_row.empty:
                country_name, indicator_name = target_row['Country Name'].values[0], target_row['Indicator Name'].values[0]
                if years:
                    target_row = target_row[years]
                else:
                    target_row = target_row.iloc[:,4:]
                target_row = target_row.dropna(axis=1)
                if not target_row.empty:
                    year_to_value = {}
                    for column in target_row:
                        year_to_value[column] = target_row[column].values[0]
                    indicator_info = {
                        "Country": country_name,
                        "Indicator Name": indicator_name,
                        "Values Per Year": year_to_value
                    }
                    
                    result_dict[indicator_id] = indicator_info
                    # Increment the counter
                    count += 1
        if count == 30:
            break
    return result_dict

In [35]:
## module to extract text from documents and return the text and document codes
def indicatorsModule(user_query):
    countries = find_mentioned_country_code(user_query)
    indicators = filter_indicators(user_query) #df, distances, indices
    years = find_target_period(user_query)
    if countries and indicators:
        # Reduce Indicator List to 2 if countries are too many
        if len(countries) > 5:
            indicators = indicators[:2]
        # for testing
        #result_structure = {}
        #result_structure["User Query"] = user_query
        #result_structure["indicatorsModule Result"] = map_to_structure(countries, indicators, years)
        result_structure = map_to_structure(countries, indicators, years)
        return result_structure
    else:
        return {}

# Test Function (indicatorsModule)

In [36]:
def calculate_scores(csv_file, moonshot_model):
    # Initialize an empty list to store processed entries
    result = []
    
    # Loop through each entry in the CSV file
    for entry in csv.DictReader(csv_file):
        print(f"Testing Query #{len(result)}...")
        query = entry['query']
        sample_answer = entry['sample_answer']
        
        # Call OpenAI for chat GPT answer
        chat_gpt_answer = callOpenAI(f""" 
                                    {query}
                                    """)
        
        # Call the moonshot model API
        moonshot_model_answer = moonshot_model(query)

        # Calculate BERT score for moonshot model answer
        P, F, R = bert_score([sample_answer], [moonshot_model_answer['answer']], lang='en', verbose=True)
        entry['moonshot_model_answer'] = moonshot_model_answer['answer']
        entry['bert_score'] = round(float(F), 2)

        # Calculate BERT score for chat GPT answer
        P, F, R = bert_score([sample_answer], [chat_gpt_answer], lang='en', verbose=True)
        entry['chat_gpt_answer'] = chat_gpt_answer
        entry['bert_score_gpt'] = round(float(F), 2)
        
        # Append the processed entry to the result list
        result.append(entry)
    
    # Return the list of processed entries
    return result

## Conduct test only using the indicator database

In [21]:
def moonshot_model_indicator_only(user_query):
    
    ##run processing modules
    indicators_dict=indicatorsModule(user_query)
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided <Sources> extract section of the JSON only to provide the correct answer to the user's query.
    
    User Query: {user_query}
    
    Sources: {indicators_dict}

    - Among the given data sources provided in the json, reference all of them, but only use the materials necessary to answer the <User Query>
    - Keep your response concise and maintain a formal academic tone. Avoid providing lengthy answers.
    - If no or only some <Sources> are provided, attempt to make suggestions or provide a partial response. If unable to obtain any answer, simply state that you don't have that information.  
    - Remove new line or tab characters from your output
    """
    
    ##synthesis module
    answer = callOpenAI(llm_instructions)

    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":indicators_dict,    
    }
    
    return response

In [23]:
# Test only using indicator database

# Specify the path to your CSV file
csv_file_path = "../testing/indicator_test/test_queries.csv"

result = []
# Open the CSV file for reading
with open(csv_file_path, mode='r') as file:
    for entry in csv.DictReader(file):
        print(f"Testing Query #{len(result)}...")
        query = entry['query']
        sample_answer = entry['sample_answer']
        
        # Call OpenAI for chat GPT answer
        chat_gpt_answer = callOpenAI(f"{query}")
        # Call the moonshot model API
        moonshot_model_answer = moonshot_model_indicator_only(query)
        
        entry['moonshot_model_answer'] = moonshot_model_answer['answer']
        entry['chat_gpt_answer'] = chat_gpt_answer
        
        # Append the processed entry to the result list
        result.append(entry)
        
# Print updated data with scores
#print(json.dumps(result, indent=4))

# Save updated data to a JSON file
with open('../testing/indicator_test/test_output_indicator_only.json', 'w') as file:
    json.dump(result, file, indent=4)

Testing Query #0...
Testing Query #1...
Testing Query #2...
Testing Query #3...


# "Layer" before implementing indicator functions

We've observed that the indicator database is necessary only when a user query requires statistical information. Otherwise, our indicator functions pull the wrong data, wasting time. We could add a layer to determine if statistical information is needed for a user query, allowing us to decide whether to implement indicator functions.

## Test Examples

In [37]:
indicator_query = "How much private investment did Ethiopia attract in its energy sector in 2021?" # Expected: True
clarification_query = "Could you clarify the role of UNDP in promoting sustainable energy solutions in urban areas?" # Expected: False
comparative_query = "How does UNDP's approach to sustainable energy differ from other international development organizations?" # Expected: False
descriptive_query = "Can you describe the impact of UNDP's sustainable energy projects on local communities?" # Expected: False
informational_query = "What are the indicators used by UNDP to measure the effectiveness of sustainable energy interventions?" # Expected: False
opinion_query = "Has UNDP's focus on sustainable energy contributed to poverty alleviation in rural areas?" # Expected: False
procedural_query = "How do you assess the feasibility of a sustainable energy project, following UNDP's guidelines?" # Expected: False
queries = "What percentage of Niger's population has access to electricity for productive use?" # Expected: True
indicator_query2 = "What percentage of China's total energy consumption in 2014 came from fossil fuels such as coal, oil, natural gas, and similar sources?" # Expected: True


test_queries = [indicator_query, clarification_query, comparative_query, descriptive_query, informational_query, opinion_query, procedural_query, queries, indicator_query2]

## First Trial - Few Shot (Didn't work well) ❌

In [38]:
# use this function to make simple openAI Calls
def callOpenAIFewShot(prompts):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=prompts
                )
    response = response_entities.choices[0].message.content
    return response

In [39]:
def indicator_layer_few_shot(user_query):
    # few-shot prompt
    llm_instructions = [
        {
            "role": "system",
            "content":"As an AI assistant, your task is to determine if a user query requires statistical information. These are questions that depend on quantitative data (such as mean, percentage, etc.) for an accurate response."
        },
        {
            "role": "user",
            "content": "What has been the average percentage of the population with access to energy, electricity, and clean cooking solutions in Haiti over the past five years?"
        },
        {
            "role": "assistant",
            "content": "True"
        },
        {
            "role": "user",
            "content": "Could you clarify the role of UNDP in promoting sustainable energy solutions in urban areas?"
        },
        {
            "role": "assistant",
            "content": "False"
        },
        {
            "role": "user",
            "content": f"{user_query}"
        }
    ]
    ##synthesis module
    answer = callOpenAIFewShot(llm_instructions)

    ##structure response
    response={
        "user_query":user_query,
        "answer":answer   
    }
    
    return response

In [40]:
for query in test_queries:
    print(indicator_layer_few_shot(query))

{'user_query': 'How much private investment did Ethiopia attract in its energy sector in 2021?', 'answer': 'True'}
{'user_query': 'Could you clarify the role of UNDP in promoting sustainable energy solutions in urban areas?', 'answer': 'True'}
{'user_query': "How does UNDP's approach to sustainable energy differ from other international development organizations?", 'answer': 'True'}
{'user_query': "Can you describe the impact of UNDP's sustainable energy projects on local communities?", 'answer': 'True'}
{'user_query': 'What are the indicators used by UNDP to measure the effectiveness of sustainable energy interventions?', 'answer': 'True'}
{'user_query': "Has UNDP's focus on sustainable energy contributed to poverty alleviation in rural areas?", 'answer': 'True'}
{'user_query': "How do you assess the feasibility of a sustainable energy project, following UNDP's guidelines?", 'answer': 'True'}
{'user_query': "What percentage of Niger's population has access to electricity for productiv

## Second Trial - Most of time works well but need to do experiments more ✅

In [15]:
def indicator_layer(user_query):
    llm_instructions = f"""
    Determine if a user query requires statistical information. These are questions that depend on quantitative data for an accurate response. Answer with "True" or False"

    User Query: {user_query}
    """
    ##synthesis module
    #answer = callOpenAIFewShot(llm_instructions)
    answer = callOpenAI(llm_instructions)

    ##structure response
    response={
        "user_query":user_query,
        "answer":answer   
    }
    
    return response

In [41]:
for query in test_queries:
    print(indicator_layer(query))

{'user_query': 'How much private investment did Ethiopia attract in its energy sector in 2021?', 'answer': 'True'}
{'user_query': 'Could you clarify the role of UNDP in promoting sustainable energy solutions in urban areas?', 'answer': 'False'}
{'user_query': "How does UNDP's approach to sustainable energy differ from other international development organizations?", 'answer': 'False'}
{'user_query': "Can you describe the impact of UNDP's sustainable energy projects on local communities?", 'answer': 'False'}
{'user_query': 'What are the indicators used by UNDP to measure the effectiveness of sustainable energy interventions?', 'answer': 'False'}
{'user_query': "Has UNDP's focus on sustainable energy contributed to poverty alleviation in rural areas?", 'answer': 'False'}
{'user_query': "How do you assess the feasibility of a sustainable energy project, following UNDP's guidelines?", 'answer': 'False'}
{'user_query': "What percentage of Niger's population has access to electricity for pro

<h2>Indicator Trello Test Format </h2>


In [3]:
wdi_csv = pd.read_csv('../data/WDI_CSV/WDICSV.csv')


In [9]:
def callOpenAIFewShot(prompts):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=prompts
                )
    response = response_entities.choices[0].message.content
    return response

In [42]:
def indicator_layer_few_shot(user_query):
    # few-shot prompt
    llm_instructions = [
        {
            "role": "system",
            "content":"As an AI assistant, your task is to determine if a user query requires statistical information. These are questions that depend on quantitative data (such as mean, percentage, etc.) for an accurate response."
        },
        {
            "role": "user",
            "content": "What has been the average percentage of the population with access to energy, electricity, and clean cooking solutions in Haiti over the past five years?"
        },
        {
            "role": "assistant",
            "content": "True"
        },
        {
            "role": "user",
            "content": "Could you clarify the role of UNDP in promoting sustainable energy solutions in urban areas?"
        },
        {
            "role": "assistant",
            "content": "False"
        },
        {
            "role": "user",
            "content": f"{user_query}"
        }
    ]
    ##synthesis module
    answer = callOpenAIFewShot(llm_instructions)

    ##structure response
    response={
        "user_query":user_query,
        "answer":answer   
    }
    
    return response

In [14]:
def indicator_trello_test(user_query) :
    is_indicator_question = indicator_layer(user_query)['answer']
    countries, indicators, years = set(), [], []
    if is_indicator_question:
        country_codes = find_mentioned_country_code (user_query)
        indicator_codes = filter_indicators(user_query)
        years = find_target_period(user_query)
        
        for country_code in country_codes:
            for indicator_code in indicator_codes:
                target_row = wdi_csv[(wdi_csv['Country Code'] == country_code) & (wdi_csv['Indicator Code'] == indicator_code)]
                if not target_row. empty:
                    country, indicator = target_row['Country Name'].values[0], target_row['Indicator Name'].values [0]
                    countries.add(country)
                    indicators.append(indicator)
                    
    query_analysis_info = {
        'is_indicator_question': is_indicator_question,
        'Countries': list(countries),
        'Indicators': indicators,
        'years': years
    }
    
    return query_analysis_info

In [43]:
indicator_query2 = "What percentage of China's total energy consumption in 2014 came from fossil fuels such as coal, oil, natural gas, and similar sources?" # Expected: True

indicator_trello_test(indicator_query2)

{'is_indicator_question': 'True',
 'Countries': ['China'],
 'Indicators': ['Electricity production from oil, gas and coal sources (% of total)',
  'Renewable energy consumption (% of total final energy consumption)',
  'Alternative and nuclear energy (% of total energy use)',
  'Fossil fuel energy consumption (% of total)',
  'Merchandise imports from low- and middle-income economies in East Asia & Pacific (% of total merchandise imports)',
  'Merchandise imports from low- and middle-income economies in Europe & Central Asia (% of total merchandise imports)',
  'Merchandise imports from low- and middle-income economies in Latin America & the Caribbean (% of total merchandise imports)',
  'Merchandise imports from low- and middle-income economies in Middle East & North Africa (% of total merchandise imports)',
  'Merchandise imports from low- and middle-income economies in South Asia (% of total merchandise imports)',
  'Merchandise imports from low- and middle-income economies in Sub-S

<h1>Trello Board AutoPopulate</h1>
<p> Automation for trello for data indicator</p>

In [44]:
import os
import csv
import asyncio
import aiohttp
import nest_asyncio
import requests
from markdownify import markdownify as md


In [45]:
# Load environment variables
trello_api_list_id = os.getenv("trello_api_list_id")
trello_api_key_id = os.getenv("trello_api_key_id")
trello_api_key_token = os.getenv("trello_api_key_token")
trello_board_id= os.getenv("trello_board_id")
trello_url ="https://api.trello.com/1/"
# print(trello_board_id)

In [46]:
#create list function - this allows for various test versions 

def create_list(list_title):
    url = f"{trello_url}lists?name={list_title}&token={trello_api_key_token}&idBoard={trello_board_id}&key={trello_api_key_id}"
    response = requests.request("POST", url)
    response_json = response.json()  # Parse response JSON
    list_id = response_json["id"]  # Access 'id' from JSON
    return list_id

#example
# list_response = create_list('User Queries')

In [47]:
#create list function - this allows for various test versions 
def create_card_label(card_id, color, name):
    url = f"{trello_url}cards/{card_id}/labels?color={color}&name={name}&token={trello_api_key_token}&idBoard={trello_board_id}&key={trello_api_key_id}"
    # print(url)
    response = requests.request("POST", url)
    response_json = response.json()  # Parse response JSON
    list_id = response_json["id"]  # Access 'id' from JSON
    return list_id

#example
# list_response = create_card_label('','')

In [48]:
def get_current_date(format='%b %d %H:%M'):
    return datetime.now().strftime(format)

In [58]:
# Function to convert DataFrame to Markdown table
def df_to_markdown(df):
    markdown = '| ' + ' | '.join(df.columns) + ' |\n'
    markdown += '| ' + ' | '.join(['---'] * len(df.columns)) + ' |\n'
    for row in df.itertuples(index=False):
        markdown += '| ' + ' | '.join(map(str, row)) + ' |\n'
    return markdown

In [60]:
#Async function to send the request

async def send_request(query,list_response,card_color,card_label_name):
    name = query["query"]
    response_indicator_layer=indicator_layer(name) 
    response = indicator_trello_test(name)
    print(response_indicator_layer)
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame({
        'Country': response['Countries'] * len(response['Indicators']),
        'Indicator': response['Indicators'],
        'Year': response['years'] * len(response['Indicators'])
    })
    markdown_table = df_to_markdown(df)

    # Convert the DataFrame to an HTML table
    # html_table = df.to_html(index=False, escape=False)

    # Print the HTML table
    desc = f""" Should have indicator: {response_indicator_layer['answer']} <br>  {markdown_table}"""
    url = f"{trello_url}cards?idList={list_response}&key={trello_api_key_id}&token={trello_api_key_token}&name={name}&desc={desc}"
    
    async with aiohttp.ClientSession() as session:
        async with session.post(url, timeout=1200) as response:
            el = ''
            # print(name)
            resp = await response.text()
            resp_dict = json.loads(resp)
            id = resp_dict['id']

            card_label_resp = create_card_label(id, card_color, card_label_name)
            print(card_label_resp)
            print("---------")



In [61]:
#Run Test process
async def mainTest():
 
    card_title = 'indicator-test' #get_current_date()
    card_color = 'red'
    card_label_name ='clarification'
    queries_source = '../testing/queries/clarification.csv'
    tasks = []
    list_response = create_list(card_title)
    with open(queries_source, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            tasks.append(send_request(row,list_response,card_color,card_label_name))
    await asyncio.gather(*tasks)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the event loop
await mainTest()


{'user_query': ' "What percentage of China\'s total energy consumption in 2014 came from fossil fuels such as coal', 'answer': 'True'}
{'user_query': ' "How much private investment did Ethiopia attract in its energy sector in 2021?"', 'answer': 'True'}
{'user_query': 'Could you clarify the role of UNDP in promoting sustainable energy solutions in urban areas?', 'answer': 'False'}
{'user_query': "What exactly does UNDP mean by 'community engagement' in the context of sustainable energy projects?", 'answer': 'False'}
{'user_query': 'Could you clarify the criteria used by UNDP to prioritize countries for renewable energy investments?', 'answer': 'False'}
{'user_query': 'What exactly are the indicators used by UNDP to measure the impact of sustainable energy interventions?', 'answer': 'False'}
{'user_query': 'Could you clarify how UNDP addresses environmental sustainability in its renewable energy initiatives?', 'answer': 'False'}
{'user_query': "What exactly does UNDP mean by 'inclusive' 